# Expel Metrics Examples
This notebook provides a few examples for how you can interact with Expel data to draw some useful insights.



**Requirements**

Environment: `python>=3.7`

Packages:
```
pyexclient>=0.0.1
pandas==4.11.0
plotly==4.11.0
python-dateutil
```

**Using this notebook**
- If needed, install the dependencies in the cell below
- Run the data acquisition cell to gather data over the specified time period
- Pick and choose following cells to see their metrics!

In [ ]:
# Run this cell if you need to install dependencies!
! pip3 install --user -i https://test.pypi.org/simple/ pyexclient>=0.0.1
! pip3 install --user pandas==1.1.2
! pip3 install --user plotly==4.11.0

# Authenticating to Workbench
Below, we're prompting you to authenticate to the Expel Workbench. This is to allow us to retrieve historical metrics that are used throught the notebook. Run the cell below, enter your credentials, and move on to the rest of the notebook!

In [ ]:
import pytz
import getpass
import pandas as pd
import plotly as py
import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from ipywidgets import VBox
from io import BytesIO
from dateutil import parser as dt_parser
from datetime import datetime, timedelta
from IPython.display import display, Markdown
from IPython.display import clear_output
from pyexclient import WorkbenchClient
from collections import defaultdict

class WorkbenchAuth:

    def __init__(self):
        self.uname = widgets.Text(
            value=None,
            placeholder='',
            description='Username:',
            disabled=False
        )
        self.passwd =  widgets.Password(
            value=None,
            placeholder='',
            description='Password:',
            disabled=False
        )
        self.mfa = widgets.Text(
            value=None,
            placeholder='123456',
            description='MFA Code:',
            disabled=False
        )
        self.login = widgets.Button(
            description='Log In',
            disabled=False,
            button_style='',
            tooltip='Log In',
            icon='check'
        )
        self.login.on_click(self.auth)
        self.widgets = [self.uname, self.passwd, self.mfa, self.login]
    
    def auth(self, b):
        self.wb = WorkbenchClient('https://workbench.expel.io', 
                         username=self.uname.value,
                         password=self.passwd.value,
                         mfa_code=self.mfa.value)
        print("Success!")

    def show(self):
        
        display(Markdown("# Authenticate to Workbench"))
        display(VBox(self.widgets))
        
auth = WorkbenchAuth()
auth.show()

# Retrieve Expel Service Metrics
In the cell below, we're retrieving Expel service metrics over your desired time range. Run the cell, enter your start/end date, and press Confirm. Once the data has been retrieved you can check out the rest of this notebook in whatever order you'd like!

In [ ]:
# Get service metrics for time period
# TODO widget this

COLUMN_MAPPING = {
    'Expel Vendor Tech Name': 'Vendor Name',
    'Insert At (T3)': 'Created At',
    'First Assigned To User At (T4)': 'First Assigned To User At',
    'First Assigned User (T4 Assignee)': 'First Assigned User ID',
    'First Assigned User Name (T4 Assignee)': 'First Assigned User Name',
    'Time Promoted To Incident': 'Promoted To Incident At',
    'Time First Investigative Action (T5a)': 'First Investigative Action At',
    'Time First Remediation Action (T5b)': 'First Remediation Action At',
    'Time First Closed (T60)': 'First Closed At',
    'Time Last Closed (T6)': 'Last Closed At',
    'First Assigned To Org At': 'First Notification At'
}

class MetricsWidget:
    
    def __init__(self, wb):
        self.wb = wb
        self.start = widgets.DatePicker(
            description='Start:',
            disabled=False,
            value=datetime.now().date() - timedelta(days=1)
        )
        self.end = widgets.DatePicker(
            description='End:',
            disabled=False,
            value=datetime.now().date()
        )
        self.button = widgets.Button(description='Confirm',
                                                disabled=False,
                                                button_style='',
                                                tooltip='Confirm',
                                                icon='check')
        self.button.on_click(self.get_metrics)
        self.widgets = [self.start, self.end, self.button]
        
    def show(self):
        display(Markdown(" # Choose date range to get metrics"))
        display(VBox(self.widgets))

    def get_metrics(self, b):
        '''
        Retrieve Expel service metrics CSV from Workbench API
        '''
        start_at = datetime(self.start.value.year, self.start.value.month, self.start.value.day)
        end_at = datetime(self.end.value.year, self.end.value.month, self.end.value.day)
        print(f"Requesting service metrics between {start_at.isoformat()} and {end_at.isoformat()}")
        url = f"/api/v2/service_metrics/export?filter[from]={start_at.isoformat()}Z&filter[to]={end_at.isoformat()}Z"
        resp = self.wb.request('get', url)
        resp.raise_for_status()
        df = pd.read_csv(BytesIO(resp.content))
        self.process_dataframe(df)

    def minutes_to_first_action(self, row):
        '''
        Determine minutes to first action
        '''
        if row['Alert Status'] == 'NEW':
            return -1
        if row['First Assigned To User At']:
            act = row['First Assigned To User At']
        elif row['First Investigated At']:
            act = row['First Investigated At']
        elif row['Promoted To Incident At']:
            act = row['Promoted To Incident At']
        elif row['First Closed At']:
            act = row['First Closed At']
        act = dt_parser.parse(act)
        ca = dt_parser.parse(row['Created At'])
        return round((act - ca).total_seconds()/60,2)

    def process_dataframe(self, df):
        '''
        Do some post processing on metrics to make the dataframe easier to use
        '''
        df = df.fillna('')
        df = df[df.Severity!='TESTING']
        df.rename(columns=COLUMN_MAPPING, inplace=True)
        df['Minutes to Action'] = df.apply(self.minutes_to_first_action, axis=1)
        df['Alert Time UTC'] = df.apply(lambda row: dt_parser.parse(row['Created At']), axis=1)
        df = df[df['Alert Time UTC']>=pd.to_datetime(str(self.start.value)).tz_localize('UTC')]
        self.df = df
        print(f"Got {len(df)} rows.")

metrics = MetricsWidget(auth.wb)
metrics.show()

# Alerting Overview
The cell below displays various statistics highlighting what happened over the time period.

In [ ]:
df = metrics.df
md = f'''
These are some totals that make up the Expel service delivery.

* Expel generated ***{len(df)}*** expel alerts
* Expel triaged and reviewed ***{len(df[(df['Investigation ID'] == "") & (df['First Notification At'] == "")])}*** alerts that required no action from the customer
* Expel performed ***{len(df[(df['Investigation Type'] == 'Investigation')]['Investigation ID'].unique())}*** investigations
* Expel identified ***{len(df[(df['Investigation Type'] == 'Incident')]['Investigation ID'].unique())}*** incidents
* Expel notified customer of activity of interest/suspicion ***{len(df[df['First Notification At'] != ""])}*** times
'''
display(Markdown(md))

# When did alerts show up?
By looking at the day of week (and time of day) that alerts show up can help show when your team experiences the most load. This can help inform staffing decisions. At Expel, we use these metrics to decide how many analysts need to be on shift at the same time. As an Expel customer, you can see how our analysts are helping to provide visibility when you don't want to be thinking about security alerts (your off hours).

**Using this cell**
- Input your Time Zone, Work Start / End Hours, and Work Days
- Review the PIE chart and heatmap to see when alerts typically show up

In [ ]:
DAYS = {
    'Monday':0,
    'Tuesday':1,
    'Wednesday':2,
    'Thursday':3,
    'Friday':4,
    'Saturday':5,
    'Sunday':6,
}

class WorkHoursWidget:
    
    def __init__(self, df):
        self.df = df
        self.timezone = widgets.Dropdown(
            options=pytz.all_timezones,
            value='UTC',
            description='Timezone:',
            disabled=False,
        )
        self.work_start = widgets.Dropdown(
            options=[i for i in range(24)],
            value=13,
            description='Work Starts:',
            disabled=False,
        )
        self.work_end = widgets.Dropdown(
            options=[i for i in range(24)],
            value=21,
            description='Work Ends:',
            disabled=False,
        )
        self.work_days = widgets.SelectMultiple(
            options=[(k,v) for k,v in DAYS.items()],
            value=[0,1,2,3,4],
            description='Work Days:',
            disabled=False
        )
        self.button = widgets.Button(description='Confirm',
                                                disabled=False,
                                                button_style='',
                                                tooltip='Confirm',
                                                icon='check')
        self.button.on_click(self.show_off_hours)
        self.widgets = [self.timezone, self.work_start, self.work_end, self.work_days, self.button]
        
    def show(self):
        display(Markdown(" # Enter your work schedule"))
        display(VBox(self.widgets))
        
    def localize(self, row):
        '''
        Localize timestamp based on time zone
        '''
        ts = dt_parser.parse(row['Created At'])
        return ts.astimezone(pytz.timezone(self.timezone.value))

    def is_off_hours(self, row):
        '''
        Determine if the alert was created during off hours
        '''
        ts = row['Alert Time Localized']
        if ts.weekday() in self.work_days.value and ts.hour >= self.work_start.value and ts.hour < self.work_end.value:
            return False
        return True

    def show_off_hours(self, b):
        clear_output()
        self.show()
        self.df['Alert Time Localized'] = self.df.apply(self.localize, axis=1)
        self.df['Is Off Hours?'] = self.df.apply(self.is_off_hours, axis=1)

        # Show pie chart for on vs off hour alerts
        pie = self.df['Is Off Hours?'].value_counts()
        fig = go.Figure(data=[go.Pie(labels=pie.index, values=pie.values)])
        fig.update_layout(title='Is Off Hours?')
        fig.show()

        # show heatmap of when alerts show up
        self.df['Hour of Day'] = self.df.apply(lambda row: row['Alert Time Localized'].hour, axis=1)
        self.df['Day of Week'] = self.df.apply(lambda row: row['Alert Time Localized'].weekday(), axis=1)
        heatmap = self.df.groupby(['Hour of Day','Day of Week'])['Expel Alert ID'].nunique().to_frame(name='Alerts').reset_index()
        h = defaultdict(dict)
        z = defaultdict(list)

        for _, row in heatmap.iterrows():
            h[row['Hour of Day']][row['Day of Week']] = row['Alerts']

        for hr in range(24):
            for day in range(7):
                z[hr].append(h[hr].get(day,0))

        fig = go.Figure(data=go.Heatmap(
                z=list(z.values()),
                x=list(DAYS.keys()),
                y=[i for i in range(24)],
                colorscale='Viridis'))

        fig.update_layout(
            title='Alert Arrival by Day of Week / Hour of Day',
            xaxis_nticks=7, yaxis_nticks=24)

        fig.show()
        
off_hrs = WorkHoursWidget(metrics.df)
off_hrs.show()

# Time to Action
Time to Action is the number of minutes it took before an analyst looked at an alert. 

In [ ]:
tmpdf = metrics.df[metrics.df.Severity!=""].groupby(['Severity'])['Minutes to Action'].agg([np.mean, 'count']).reset_index()
md = "**Alert Response Time by Severity**\n"
for _, row in tmpdf.iterrows():
    md += f'* Expel saw ***{row["count"]}*** {row["Severity"]} alerts, '
    md += f'on average they were looked at in ***{row["mean"]:.03f}*** minutes\n'

display(Markdown(md))

fig = go.Figure([go.Bar(x=tmpdf.Severity, y=tmpdf["mean"])])
fig.update_layout(title="Alert Response Times by Severity",
                  xaxis_title="Alert Severity",
                  yaxis_title="Avg. Minutes to Action")
fig.show()

# Top Asset Values in Expel Alerts
The below cell will gather stack the top users, hosts and Operating Systems observed in Expel Alerts for the time period. This can help you get a sense for which assets are repeat offenders.

In [ ]:
# Retrieve additional evidence and display top values in PIE charts
hostnames = defaultdict(int)
usernames = defaultdict(int)
operatingsystems = defaultdict(int)
d_ips = defaultdict(int)
errors = []

def flatten_json(y):
    out = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '.')
        elif type(x) is list:
            for i, a in enumerate(x):
                flatten(a, name + str(i) + '.')
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

def get_os_name(ev):
    for key, val in flatten_json(ev).items():
        if key.endswith('os.name'):
            return val
    return None

print(f"Retrieving evidence for {len(metrics.df)} alerts... this could take a few minutes")
i = 0
for alert_id in metrics.df['Expel Alert ID'].unique():
    if i > 100:
        print("Stopping after 100 alerts!")
        break
    try:
        ea = auth.wb.expel_alerts.get(id=alert_id)
    except:
        errors.append(f"Error retreiving evidence for expel alert {alert_id}")
        continue
    found_hn = set()
    found_un = set()
    found_os = set()
    found_dips = set()
    for ev in ea.evidence:
        if ev.evidence_type == 'HOSTNAME':
            if ev.evidence not in found_hn:
                hostnames[ev.evidence] += 1
                found_hn.add(ev.evidence)
        elif ev.evidence_type == 'USERNAME':
            if ev.evidence not in found_un:
                usernames[ev.evidence] += 1
                found_un.add(ev.evidence)
        elif ev.evidence_type == 'DST_IP':
            if ev.evidence not in found_dips:
                d_ips[ev.evidence] += 1
                found_dips.add(ev.evidence)
    for va in ea.vendor_alerts:
        os = get_os_name(va.evidence_summary)
        if os not in found_os:
            operatingsystems[os] += 1
            found_hn.add(os)
    i += 1
print(f"Retrieved evidence with {len(errors)} errors")

if hostnames:
    fig1 = go.Figure(data=[go.Pie(labels=list(hostnames.keys()), values=list(hostnames.values()))])
    fig1.update_layout(title='Top Host Names')
    fig1.show()
else:
    print("No hostnames found in Expel Alerts")

if usernames:
    fig2 = go.Figure(data=[go.Pie(labels=list(usernames.keys()), values=list(usernames.values()))])
    fig2.update_layout(title='Top User Names')
    fig2.show()
else:
    print("No usernames found in Expel Alerts")

if operatingsystems:
    fig3 = go.Figure(data=[go.Pie(labels=list(operatingsystems.keys()), values=list(operatingsystems.values()))])
    fig3.update_layout(title='Top Operating Systems')
    fig3.show()
else:
    print("No operating systems found in Expel Alerts")
    
if d_ips:
    fig4 = go.Figure(data=[go.Pie(labels=list(d_ips.keys()), values=list(d_ips.values()))])
    fig4.update_layout(title='Top Destination IPs')
    fig4.show()
else:
    print("No destination IPs found in Expel Alerts")

# Change point analysis
Change point analysis allows you to spot statistically significant, sustained changes in timeseries data. Statistically significant just means that it's a change that is beyond the typical daily fluctuation and big enough that we should care about it.  For example, say we normally see around 100 Expel Alerts a day but suddenly we start seeing 300 a day. The first day we saw 300 would be considered a "change point" indicating that something has caused our metrics to behave differently. This kind of analysis can be really useful to pinpoint when things change and spur further investigation into the root cause.

In [ ]:
import random
import sys

def cumsums(data):
    """
    Description:    Calculates the cumulative sums 
    Arguments:      data - a list of floats
    """
    
    series_average = np.average(data)
    csums = []
    csums.append(0)
    
    for i in range(0, len(data)):
        csums.append( (data[i] - series_average) + csums[i])
    return csums    

def randomize(data):
    temp_list = []
    temp_list.extend(data)
    
    result = []
    for i in range(len(temp_list)):
        element = random.choice(temp_list)
        temp_list.remove(element)
        result.append(element)
    return result
    
def bootstrap(data, iterations=1000):
    """
    Description:    used, along with the confidence interval,
                    to detect if a change occurred int he series.
                    Creates 100 bootsrapped series, shuffles the original data list
                    then calculates the cumulative sum for each shuffled data series
    Arguments:      data - a list of floats
    Returns:        returns the confidence interval
    """
    
    cumsum_original = cumsums(data)
    sdiff_original = max(cumsum_original) - min(cumsum_original)
    
    #boot strap n samples
    bootstrapped_series = [randomize(data) for i in range(iterations)]
    
    #find cumumlative sums for the bootstrapped samples
    bootstrapped_cumsums = [cumsums(bootstrapped_series[i]) for i in range(len(bootstrapped_series))]
    x = [max(bootstrapped_cumsums[i]) - min(bootstrapped_cumsums[i]) for i in range(len(bootstrapped_series))]
    
    #find the number of bootstrapped series where
    #S_diff is < S_diff of the original series
    n = 0
    for i in range(len(x)):
        if (x[i] < sdiff_original):
            n = n + 1    

    s =  (n/float(iterations)) * 100.0
    return s
    
def find_index_of_maximum(cumsum):
    """
    Description:  Find the index of the maximum value from the cummulative sums
    """
    max_number = sys.float_info.min
    max_index = 0
    abs_vals = [abs(x) for x in cumsum]

    for (i, num) in enumerate(abs_vals):
        if num > max_number:
            max_number = num
            max_index = i
        
    return max_index

def get_changepoints(data, change_points, confidence_level = 95, offset = 0):
    """
    Description:    Call the function by passing a data series 
                    Once a change has been detected, break the data into two segments,
                    one each side of the change-point, and the analysis repeated for each segment.                   
    Returns:        Indexes of change points detected in the data series
    """
    if not change_points:
        change_points = []

    confidence = bootstrap(data, 1000)
    if (confidence > confidence_level):
        cumsum = cumsums(data)
        max_index = find_index_of_maximum(cumsum)

        #add change point found to list
        #use offset to find the correct index based on original data
        change_points.extend([max_index + offset])

        #split the data into two, and calculate change points
        get_changepoints(data[:max_index], change_points, confidence_level, offset)
        get_changepoints(data[max_index:], change_points, confidence_level, offset + max_index - 1)

    return change_points

def ff_ts_with_changepoints(x_vals, y_vals, change_points, title):
    """
    Create a figure of a timeseries with changepoints marked.
    """
    layout = {
            'title' : title,
            'showlegend' : False,
            'shapes' : [ ] 
    }

    last = 0
    cpoint_height = max(y_vals)
    for i in change_points:
        cpoint = x_vals[i]
        # Add a vertical line for the changepoint location.
        layout['shapes'].append(
                    {
                        'type': 'line',
                        'x0': cpoint,
                        'y0': 0,
                        'x1': cpoint,
                        'y1': cpoint_height,
                        'line': {
                            'color': 'rgb(255,0,0)',
                            'width': 4,
                            'dash': 'dot',
                        },
                    })
        # Add a horizontal line for the mean in this regime.
        new_mean = np.mean(y_vals[last:i])
        print("Mean: {}".format(new_mean))
        layout['shapes'].append(
                    {
                        'type': 'line',
                        'x0': x_vals[last],
                        'y0': new_mean,
                        'x1': cpoint,
                        'y1': new_mean,
                        'line': {
                            'color': 'rgb(211,211,0)',
                            'width': 4,
                            'dash': 'dot',
                        },
                    })
        last = i

    new_mean = np.mean(y_vals[last:-1])
    print("Mean: {}".format(new_mean))
    layout['shapes'].append(
                {
                    'type': 'line',
                    'x0': x_vals[last],
                    'y0': new_mean,
                    'x1': x_vals[-1],
                    'y1': new_mean,
                    'line': {
                        'color': 'rgb(211,211,0)',
                        'width': 4,
                        'dash': 'dot',
                    },
                })
    layout['shapes'].append(
                {
                    'type': 'line',
                    'x0': x_vals[0],
                    'x1': x_vals[-1],
                    'line': {
                        'color': 'gray',
                        'width': 4,
                        'dash': 'dashdot',
                    },

                })
 
    
    trace = go.Scatter(
            x = x_vals,
            y = y_vals,
            name = "Events"
       )
    
    data = [trace]
    
    fig = dict(data=data, layout=layout)
  
    return fig

change_points = []
temp = metrics.df.copy()
temp['Date'] = temp.apply(lambda row: dt_parser.parse(row['Created At']).date(), axis=1)
temp = temp.groupby(['Date'])['Expel Alert ID'].nunique().to_frame(name='alert_count').reset_index()
temp = temp.set_index('Date')
idx = pd.date_range(temp.index[0], temp.index[-1])
temp = temp.reindex(idx, fill_value=0)
points = get_changepoints(list(temp.alert_count), change_points)
points.sort()
print("Change Points: ", points)

#Graph changepoints
#plt.figure(figsize = (10,5))
fig = ff_ts_with_changepoints(temp.index, temp.alert_count, points, 'Expel Alert Timeseries w/ Change Points')
f = go.Figure(fig)
f.show()